## 0. Pré-Processamento

### 0.1 Importando a(s) biblioteca(s) e o dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('scrapping_roupas_marketplace.csv')

## 1. Limpeza da base

In [4]:
#Removendo a coluna indesejada
df = df.drop('Unnamed: 0', axis=1)

### 1.1 Tratando os nulos

Os nulos na categoria de nota não foram tratados e o trabalho para ajuste deve ficar a cargo de quem trabalhar com a base.

In [5]:
df.isna().sum()

Título             0
Reais              0
Centavos         243
Condição           0
Nota             648
N_Avaliações     648
Desconto        1333
Marca             23
Material         875
Gênero            87
Temporada       1072
Review1          692
Review2          853
Review3          974
dtype: int64

In [6]:
df[['Centavos', 'N_Avaliações', 'Desconto']] = df[['Centavos', 'N_Avaliações', 'Desconto']].fillna(0)
df[['Marca', 'Material', 'Gênero', 'Temporada', 'Review1', 'Review2', 'Review3']] =df[['Marca', 'Material', 'Gênero', 'Temporada', 'Review1', 'Review2', 'Review3']].fillna('N/A')

In [7]:
df[['Reais', 'Centavos']] = df[['Reais', 'Centavos']].astype(int).astype(str)
df['Preco'] = (df['Reais'] + '.' + df['Centavos']).astype(float)

In [8]:
df = df.drop(['Reais', 'Centavos'], axis=1)

### 1.1 Arrumando colunas

In [9]:
#Ajeitando as colunas do número de avaliações, tirando os ()
df['N_Avaliações'] = df['N_Avaliações'].apply(lambda x: int(x) if x == 0 else int(x[1:-1]))

In [10]:
#Arrumando as colunas de desconto, transformando em número
df['Desconto'] = df['Desconto'].apply(lambda x: int(str(x).replace('% OFF', ''))/100)

In [12]:
#Utilizando a coluna de "Condição" para identificar o número de itens vendidos em cada anúncio
dic_condicoes = {'Novo': 0, 
                 '1 vendido': 1,
                 '2': 2,
                 '3': 3,
                 '4': 4,
                 '+5': 5,
                 '+25': 25,
                 '+50': 50,
                 '+100': 100,
                 '+500': 500,
                 '+1000': 1000,
                 '+5mil': 5000,
                 '+10mil': 10000,
                 '+50mil': 50000,
                 }

df['Condição'] = df['Condição'].apply(lambda x: x.replace('Novo  |  ', ''))
df['Condição'] = df['Condição'].apply(lambda x: x.replace(' vendidos', ''))

df['n_vendidos_categoria'] = df['Condição'].replace(dic_condicoes)

In [13]:
df = df.drop('Condição', axis=1)

## 2. Salvando a base

In [15]:
df.to_csv('dataset_roupas_marketplacebr_limpo.csv')